In [ ]:
# --- install (first time) ---
# pip install pygmt richdem numpy xarray

import pygmt
import numpy as np

# A small padded box around Montserrat (tweak as you like)
# Montserrat ~ 16.70°N, 62.20°W
region = [-62.30, -61.95, 16.60, 16.90]  # [west, east, south, north]

# Cut 1-arcsecond DEM for the region
dem = pygmt.grdcut("@earth_relief_01s", region=region)

# Optional: clip extreme outliers (rare) to keep hydrology stable
q1, q99 = np.nanpercentile(dem.data, [1, 99])
dem = dem.where(dem < (q99 + 5*(q99-q1)))

# Save a GeoTIFF (uses GMT’s writers under the hood)
# (the "=gd:GTiff" suffix tells GMT to write GeoTIFF)
pygmt.grdconvert(
    grid=dem,
    outgrid="montserrat_01s.tif=gd:GTiff",
    convert="=gd:GTiff"
)

print("Wrote montserrat_01s.tif")


In [ ]:
# --- extra install if needed ---
# pip install rioxarray

import pygmt
#import rioxarray

# Define region around Montserrat
region = [-62.30, -61.95, 16.60, 16.90]

# Fetch DEM (1 arc-second)
dem = pygmt.grdcut("@earth_relief_01s", region=region)

# Attach CRS info so rioxarray can write GeoTIFF
dem.rio.write_crs("EPSG:4326", inplace=True)

# Save to GeoTIFF
dem.rio.to_raster("montserrat_01s.tif")

print("Wrote montserrat_01s.tif")


In [ ]:
import whitebox
wbt = whitebox.WhiteboxTools()

# fill depressions
wbt.fill_depressions("montserrat_01s.tif", "montserrat_filled.tif")

# d8 flow accumulation
wbt.d8_flow_accumulation("montserrat_filled.tif", "montserrat_fa.tif", out_type="cells")

# extract streams with threshold (e.g. 200 cells)
wbt.extract_streams("montserrat_fa.tif", "montserrat_streams.tif", threshold=200)


In [ ]:
import numpy as np
import richdem as rd

# Load GeoTIFF
dem = rd.LoadGDAL("montserrat_01s.tif")

# Fill depressions (carves/levels small pits that stall flow)
filled = rd.FillDepressions(dem, in_place=False)

# D8 flow accumulation (number of upstream cells, float64)
fa = rd.FlowAccumulation(filled, method='D8')

# --- Convert drainage-area thresholds (km²) to "cells" at 16.7°N ---
# Approx 1″ cell size at 16.7°N:
#   NS ≈ 30.9 m, EW ≈ 29.6 m  → cell_area ≈ 30.9*29.6 ≈ 915 m²
cell_area_m2 = 30.9 * 29.6  # ≈ 915 m²
def cells_for_km2(km2):
    return (km2*1_000_000) / cell_area_m2

# Choose three thresholds to try (tune later)
thresholds_km2 = [0.1, 0.25, 0.5]  # 0.1–0.5 km²
thresholds_cells = [int(cells_for_km2(k)) for k in thresholds_km2]
print("FA thresholds (cells):", dict(zip(thresholds_km2, thresholds_cells)))

# Make binary stream rasters for each threshold
streams_maps = {}
for k, n_cells in zip(thresholds_km2, thresholds_cells):
    streams = (fa > n_cells).astype(np.uint8)
    outname = f"streams_{str(k).replace('.','p')}km2.tif"
    rd.SaveGDAL(outname, streams, prototype=dem)
    streams_maps[k] = outname
    print("Wrote", outname)

# Also save hillshade for quick look
hs = rd.TerrainAttribute(filled, attrib='hillshade')
rd.SaveGDAL("hillshade.tif", hs, prototype=dem)
print("Wrote hillshade.tif")


In [ ]:
import pygmt

fig = pygmt.Figure()

# Base hillshade (stretches automatically)
fig.grdimage("hillshade.tif", cmap="gray", shading=True, frame=["af", "WSne"])

# Overlay streams for the three thresholds (thicker for larger basins)
colors = ["blue", "dodgerblue3", "navy"]  # tweak if you like
pens = ["0.6p", "0.9p", "1.2p"]

for (k, tif), c, p in zip(streams_maps.items(), colors, pens):
    # Convert raster streams to vectors with contour at value=0.5 to get lines
    # (Alternative: gdal_polygonize + centerlines, but this keeps it GMT-native)
    fig.grdcontour(
        grid=tif,
        interval=1,              # draws where value crosses 1 (binary raster)
        pen=p + "," + c,
        limit=[0.5, 1.5],        # ensure only the “1” features are traced
        annotation=None
    )

fig.colorbar(position="JMR+w5c/0.4c+o0.6c/0c", frame='af+l"hillshade"')
fig.basemap(map_scale="jBL+w5k+o0.5c/0.8c", frame=True)

fig.savefig("montserrat_streams.png", dpi=300)
print("Wrote montserrat_streams.png")


In [ ]:
# If needed:
# conda install -c conda-forge pygmt rioxarray whitebox
# (pip is ok too: pip install pygmt rioxarray whitebox)

import os
from pathlib import Path
import pygmt
import numpy as np
import rioxarray
from whitebox import WhiteboxTools

# 1) Paths (use absolute to avoid "No such file" panics)
outdir = Path.cwd() / "wbt_montserrat"
outdir.mkdir(exist_ok=True)
dem_path      = outdir / "montserrat_01s.tif"
filled_path   = outdir / "montserrat_filled.tif"
fa_path       = outdir / "montserrat_fa.tif"
streams_path  = outdir / "montserrat_streams.tif"

# 2) Fetch 1″ DEM and write a clean GeoTIFF with explicit NoData
region = [-62.30, -61.95, 16.60, 16.90]  # Montserrat + padding
dem = pygmt.grdcut("@earth_relief_01s", region=region)

# Make sure dtype + CRS + NoData are set; keep ocean as NoData
dem = dem.astype("float32")
dem.rio.write_crs("EPSG:4326", inplace=True)
dem.rio.write_nodata(-9999.0, inplace=True)
dem.rio.to_raster(dem_path.as_posix())

print(f"Wrote DEM: {dem_path}")

# 3) WhiteboxTools: set working dir & run tools with absolute paths
wbt = WhiteboxTools()
wbt.set_working_dir(outdir.as_posix())
wbt.verbose = True

# Fill depressions
wbt.fill_depressions(
    dem=dem_path.as_posix(),
    output=filled_path.as_posix(),
    fix_flats=True
)

# D8 flow accumulation
wbt.d8_flow_accumulation(
    dem=filled_path.as_posix(),
    output=fa_path.as_posix(),
    out_type="cells"
)

# Extract streams
wbt.extract_streams(
    flow_accum=fa_path.as_posix(),
    output=streams_path.as_posix(),
    threshold=275
)



print("Done. Outputs in:", outdir)


In [ ]:
from whitebox import WhiteboxTools
from pathlib import Path

outdir = Path.cwd() / "wbt_montserrat"
outdir.mkdir(exist_ok=True)
dem_path      = outdir / "montserrat_01s.tif"
filled_path   = outdir / "montserrat_filled.tif"
fa_path       = outdir / "montserrat_fa.tif"
streams_path  = outdir / "montserrat_streams.tif"

wbt = WhiteboxTools()
wbt.set_working_dir(outdir.as_posix())
wbt.verbose = True

# Fill depressions
wbt.run_tool(
    "FillDepressions",
    {
        "dem": dem_path.as_posix(),
        "output": filled_path.as_posix(),
        "fix_flats": True
    }
)

# D8 flow accumulation
wbt.run_tool(
    "D8FlowAccumulation",
    {
        "dem": filled_path.as_posix(),
        "output": fa_path.as_posix(),
        "out_type": "cells"
    }
)

# Extract streams
wbt.run_tool(
    "ExtractStreams",
    {
        "flow_accum": fa_path.as_posix(),
        "output": streams_path.as_posix(),
        "threshold": 275
    }
)

print("Done. Outputs in", outdir)


In [ ]:
import pygmt
from pathlib import Path

outdir = Path.cwd() / "wbt_montserrat"
dem_path = outdir / "montserrat_01s.tif"
streams_path = outdir / "montserrat_streams.tif"

# 1) Nice-looking hillshade from the DEM
#    (multi-azimuth gives more relief; tweak if you like)
hs315 = pygmt.grdgradient(dem_path.as_posix(), azimuth=315, normalize="e0.7")
hs045 = pygmt.grdgradient(dem_path.as_posix(), azimuth=45,  normalize="e0.7")
hs = (hs315 + hs045) / 2  # simple blend in xarray

# 2) Build the map
fig = pygmt.Figure()

# Base grayscale hillshade
fig.grdimage(hs, cmap="gray", frame=["WSne", "af"])

# Semi-transparent elevation tint (optional; comment out if you prefer gray only)
fig.grdimage(dem_path.as_posix(), cmap="geo", transparency=50)

# 3) Overlay streams (binary raster) as lines
#    We trace the 0→1 boundary with grdcontour to get “lines”
fig.grdcontour(
    grid=streams_path.as_posix(),
    interval=1,
    limit=[0.5, 1.5],
    pen="1.1p,blue"
)

# Extras: coastline, scalebar, and a neat legend title
fig.coast(shorelines="0.25p,black")
fig.basemap(map_scale="jBL+w5k+o0.7c/0.8c", compass="jTR+w1.2c+o0.8c/0.8c+f")
fig.text(x=0.02, y=0.98, text="Streams (threshold ≈0.25 km²)", region="0/1/0/1",
         font="10p,Helvetica-Bold,blue", justify="TL", N=True, offset="0c/0c")

# 4) Save
fig.savefig(outdir / "montserrat_streams_overlay.png", dpi=300)
fig.savefig(outdir / "montserrat_streams_overlay.pdf")
print("Wrote:", outdir / "montserrat_streams_overlay.png")


In [ ]:
from pathlib import Path
outdir = Path.cwd()/ "wbt_montserrat"
for p in sorted(outdir.glob("*")):
    print(p.name)
print("streams exists?", (outdir/"montserrat_streams.tif").exists())
